## Part 1: Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

2025-02-05 12:03:23.757223: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [2]:
# Determine the number of unique values in each column
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [3]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]

In [44]:
# Create a list of at least 10 column names to use as X data
selected_columns = [
    'Age', 'DistanceFromHome', 'Education', 'JobLevel', 
    'JobSatisfaction', 'NumCompaniesWorked', 'PercentSalaryHike', 
    'TotalWorkingYears', 'TrainingTimesLastYear', 'YearsAtCompany'
]

# Create X_df using your selected columns
X_df = attrition_df[selected_columns]

# Show the data types for X_df
X_df.dtypes

Age                      int64
DistanceFromHome         int64
Education                int64
JobLevel                 int64
JobSatisfaction          int64
NumCompaniesWorked       int64
PercentSalaryHike        int64
TotalWorkingYears        int64
TrainingTimesLastYear    int64
YearsAtCompany           int64
dtype: object

In [45]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size=0.2, random_state=1)

In [46]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary
X_df_encoded = pd.get_dummies(X_df)

In [47]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
X_train_scaled = scaler.fit_transform(X_train)

# Scale the training and testing data
X_test_scaled = scaler.transform(X_test)

In [52]:
from sklearn.preprocessing import OneHotEncoder

# Create a OneHotEncoder for the Department column
encoder = OneHotEncoder(sparse_output=False, handle_unknown="ignore")

# Fit the encoder to the training data
y_train_encoded = encoder.fit_transform(y_train[['Department']])
y_test_encoded = encoder.transform(y_test[['Department']])

# Create two new variables by applying the encoder
# to the training and testing data
y_train_encoded_df = pd.DataFrame(y_train_encoded, columns=encoder.get_feature_names_out(['Department']))
y_test_encoded_df = pd.DataFrame(y_test_encoded, columns=encoder.get_feature_names_out(['Department']))
y_train_final = pd.concat([y_train.reset_index(drop=True).drop("Department", axis=1), y_train_encoded_df], axis=1)
y_test_final = pd.concat([y_test.reset_index(drop=True).drop("Department", axis=1), y_test_encoded_df], axis=1)

In [53]:
# Create a OneHotEncoder for the Attrition column
encoder = OneHotEncoder(sparse_output=False, handle_unknown="ignore")

# Fit the encoder to the training data
y_train_encoded = encoder.fit_transform(y_train[['Attrition']])
y_test_encoded = encoder.transform(y_test[['Attrition']])

# Create two new variables by applying the encoder
# to the training and testing data
y_train_encoded_df = pd.DataFrame(y_train_encoded, columns=encoder.get_feature_names_out(['Attrition']))
y_test_encoded_df = pd.DataFrame(y_test_encoded, columns=encoder.get_feature_names_out(['Attrition']))
y_train_final = pd.concat([y_train.reset_index(drop=True).drop("Attrition", axis=1), y_train_encoded_df], axis=1)
y_test_final = pd.concat([y_test.reset_index(drop=True).drop("Attrition", axis=1), y_test_encoded_df], axis=1)

## Part 2: Create, Compile, and Train the Model

In [61]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Find the number of columns in the X training data.
input_dim = X_train_scaled.shape[1]

# Create the input layer
model_input = Sequential()
model_input.add(Dense(units=64, activation='relu', input_dim=input_dim))

# Create at least two shared layers
model_input.add(Dense(units=32, activation='relu'))
model_input.add(Dense(units=16, activation='relu'))
model_input.add(Dense(1, activation='sigmoid'))

/opt/anaconda3/envs/tsenv/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [60]:
from tensorflow.keras.layers import Input


# Create a branch for Department
# with a hidden layer and an output layer
department_input = Input(shape=(X_train_encoded.shape[1],))

# Create the hidden layer
department_hidden = Dense(32, activation='relu')(department_input)

# Create the output layer
department_output = Dense(1, activation='sigmoid')(department_hidden)


In [59]:
# Create a branch for Attrition
# with a hidden layer and an output layer
attrition_input = Input(shape=(X_train_encoded.shape[1],))

# Create the hidden layer
attrition_hidden = Dense(32, activation='relu')(attrition_input)

# Create the output layer
attrition_output = Dense(1, activation='sigmoid')(attrition_hidden)

In [64]:
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Create the model
model = Model(inputs=[attrition_input, department_input], outputs=[department_output, attrition_output])

# Compile the model
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# Summarize the model
model.summary()

Model: "functional_8"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4       │ (None, 4)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_3       │ (None, 4)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_10 (Dense)    │ (None, 32)        │        160 │ input_layer_4[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_8 (Dense)     │ (None, 32)        │        160 │ input_layer_3[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_11 (Dense)    │ (None, 1)         │         33 │ dense_10[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_9 (Dense)     │ (None, 1)         │         33 │ dense_8[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 386 (1.51 KB)

 Trainable params: 386 (1.51 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Train the model
history = model.fit(
    [X_train_scaled],  
    [y_train_final],               
    epochs=100,                                             
    batch_size=32,                                         
    validation_data=([X_test_scaled], [y_test_final])
)

In [ ]:
# Evaluate the model with the testing data
test_loss, test_accuracy = model.evaluate(
    [X_test_scaled],  
    [y_test_final],
)

In [ ]:
# Print the accuracy for both department and attrition
department_accuracy = history.history['accuracy'][0] 
attrition_accuracy = history.history['accuracy'][1] 

print(f"Department Accuracy: {department_accuracy}")
print(f"Attrition Accuracy: {attrition_accuracy}")

Attrition predictions accuracy: 0.7880434989929199
Department predictions accuracy: 0.5


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. No because of imbalanced classes and loss functions.
2. Sigmoid for binary classification and Softmax for multiclass classification.
3. Feature Engineering/Hyperparameter Tuning/Additional Layers